In [1]:
!pip install google-cloud-storage

In [4]:

import os
import json
import imageio
import cv2
import numpy as np
from io import BytesIO
from skimage.metrics import structural_similarity as ssim
from google.cloud import storage
from google.colab import auth
import tempfile

auth.authenticate_user()

# Set up Google Cloud Storage client
storage_client = storage.Client()
bucket_name = 'gif-bucket-1000'
bucket = storage_client.bucket(bucket_name)

def resize_and_normalize_frame(frame, target_size=(128, 128)):
    """Resize and normalize the frame while preserving aspect ratio."""
    original_height, original_width = frame.shape[:2]
    aspect_ratio = original_width / original_height

    if aspect_ratio > 1:  # Wider than tall
        new_width = target_size[0]
        new_height = int(target_size[0] / aspect_ratio)
    else:  # Taller than wide or square
        new_width = int(target_size[1] * aspect_ratio)
        new_height = target_size[1]

    resized_frame = cv2.resize(frame, (new_width, new_height))

    # Calculate padding
    delta_w = target_size[0] - new_width
    delta_h = target_size[1] - new_height
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    # Add padding to maintain the target size
    normalized_frame = cv2.copyMakeBorder(resized_frame, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    normalized_frame = normalized_frame.astype(np.float32) / 255.0

    return normalized_frame

def augment_frame(frame):
    """Apply random data augmentation to the frame."""
    if np.random.rand() < 0.5:
        frame = cv2.flip(frame, 1)  # Horizontal flip
    if np.random.rand() < 0.5:
        rows, cols = frame.shape[:2]
        rotation_angle = np.random.randint(-10, 10)
        M = cv2.getRotationMatrix2D((cols/2, rows/2), rotation_angle, 1)
        frame = cv2.warpAffine(frame, M, (cols, rows))
    return frame

def extract_key_frames(frames, ssim_threshold=0.95, min_scene_change=10):
    """Extract key frames based on structural similarity."""
    key_frames = []
    prev_frame = None
    scene_change_counter = 0
    scene_changes = []  # Initialize scene_changes here

    for i, frame in enumerate(frames):
        if not isinstance(frame, np.ndarray):
            print(f"Frame {i}: Not a valid NumPy array.")
            continue

        if frame.size == 0 or len(frame.shape) != 3:
            print(f"Frame {i}: Empty or has invalid dimensions: {frame.shape if isinstance(frame, np.ndarray) else 'N/A'}")
            continue

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

        if prev_frame is None:
            key_frames.append(frame)
            prev_frame = gray_frame
            continue

        similarity_score = ssim(prev_frame, gray_frame, data_range=1.0)

        if similarity_score < ssim_threshold:
            scene_change_counter += 1
            if scene_change_counter >= min_scene_change:
                key_frames.append(frame)
                scene_changes.append(i)  # Add index to scene_changes
                scene_change_counter = 0
        else:
            scene_change_counter = 0

        prev_frame = gray_frame

    return key_frames, scene_changes  # Return scene_changes


def preprocess_gif(gif_file_name, gif_id):
    """Preprocesses a GIF from GCS, extracts key frames, and uploads them to the bucket."""
    try:
        gif_blob = bucket.blob(os.path.join('gifs', gif_file_name))
        gif_bytes = gif_blob.download_as_bytes()

        with imageio.get_reader(BytesIO(gif_bytes), 'gif') as reader:
            frame_count = reader.get_length()
            try:
                frame_rate = 1 / reader.get_meta_data()['duration']
            except (KeyError, AttributeError, ZeroDivisionError):
                frame_rate = 10

            processed_frames = []
            for frame in reader:
                if frame.shape[-1] == 4:
                    frame = cv2.cvtColor(frame, cv2.COLOR_RGBA2RGB)
                resized_frame = resize_and_normalize_frame(frame)
                augmented_frame = augment_frame(resized_frame)
                processed_frames.append(augmented_frame)

        key_frames, scene_changes = extract_key_frames(processed_frames)

        frame_paths = []
        for i, frame in enumerate(key_frames):
            # Construct the path for the frame in the bucket
            frame_path = os.path.join('gif_frames', str(gif_id), f'frame_{i}.png')

            # Create a new blob and upload the frame
            frame_blob = bucket.blob(frame_path)
            _, temp_local_filename = tempfile.mkstemp()
            #cv2.imwrite(temp_local_filename, frame * 255)
            frame = (frame * 255).astype(np.uint8)
            imageio.imwrite(temp_local_filename, frame, format='PNG')
            frame_blob.upload_from_filename(temp_local_filename)
            os.remove(temp_local_filename)

            frame_paths.append(frame_path)  # Store the GCS path

        gif_data = {
            'gif_id': gif_id,
            'gif_file': gif_file_name,
            'file_size': len(gif_bytes),  # Get file size from gif_bytes
            'frame_count': frame_count,
            'frame_rate': frame_rate,
            'frames': frame_paths,  # Store paths to frames in GCS
            'scene_changes': scene_changes,
        }

        return gif_data

    except Exception as e:
        print(f"Error processing GIF {gif_file_name}: {e}")
        return None

def process_gifs(file_path):
    """Processes GIFs listed in a text file."""
    gif_data_list = []
    gif_id_counter = 1
    try:
        # Download the file from GCS
        blob = bucket.blob(file_path)
        file_content = blob.download_as_string().decode('utf-8')

        for line in file_content.splitlines():
            gif_file_name = line.strip()
            gif_data = preprocess_gif(gif_file_name, gif_id_counter)
            if gif_data:
                gif_data_list.append(gif_data)
                gif_id_counter += 1
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

    return gif_data_list

# Process GIFs for each data split
train_data = process_gifs('train.txt')
val_data = process_gifs('val.txt')
test_data = process_gifs('test.txt')

# Upload the data to JSON files in GCS
def upload_json_to_gcs(data, filename):
    """Uploads a JSON object to GCS."""
    try:
        json_blob = bucket.blob(filename)
        json_blob.upload_from_string(json.dumps(data, indent=4), content_type='application/json')
        print(f"Uploaded {filename} to GCS")
    except Exception as e:
        print(f"Error uploading {filename} to GCS: {e}")

upload_json_to_gcs(train_data, 'train_data.json')
upload_json_to_gcs(val_data, 'val_data.json')
upload_json_to_gcs(test_data, 'test_data.json')

Error processing GIF 53556.gif: No packer found from P to L
Frame 0: Empty or has invalid dimensions: (128, 128)
Frame 1: Empty or has invalid dimensions: (128, 128)
Frame 2: Empty or has invalid dimensions: (128, 128)
Frame 3: Empty or has invalid dimensions: (128, 128)
Frame 4: Empty or has invalid dimensions: (128, 128)
Frame 5: Empty or has invalid dimensions: (128, 128)
Frame 6: Empty or has invalid dimensions: (128, 128)
Frame 7: Empty or has invalid dimensions: (128, 128)
Frame 8: Empty or has invalid dimensions: (128, 128)
Frame 9: Empty or has invalid dimensions: (128, 128)
Frame 10: Empty or has invalid dimensions: (128, 128)
Frame 11: Empty or has invalid dimensions: (128, 128)
Frame 12: Empty or has invalid dimensions: (128, 128)
Frame 13: Empty or has invalid dimensions: (128, 128)
Frame 14: Empty or has invalid dimensions: (128, 128)
Frame 15: Empty or has invalid dimensions: (128, 128)
Frame 16: Empty or has invalid dimensions: (128, 128)
Frame 17: Empty or has invalid d